This notebook presupposes an OpenAI "clone" server running on localhost.

We used VLLM to create a local server using the Llama3 model.

In particular, we used the following terminal command to initialize the server:

```python -m vllm.entrypoints.openai.api_server --model meta-llama/Meta-Llama-3-8B-Instruct --dtype half ```

In [ ]:
import pandas as pd
import openai
from pandas_parallel_apply import DataFrameParallel

In [ ]:
isarcasm = pd.read_csv("train.En.csv")

In [ ]:
print(len(isarcasm))

3468


In [ ]:
isarcasm.head(2)

,Unnamed: 0,tweet,sarcastic,rephrase,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffein...,1,"College is really difficult, expensive, tiring...",0.0,1.0,0.0,0.0,0.0,0.0
1,1,I love it when professors draw a big question ...,1,I do not like when professors don’t write out ...,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def get_sarcasm_type(row):
    sarcasm_types = []
    columns = ['rephrase','sarcasm','irony','satire', 'understatement', 'overstatement','rhetorical_question']
    for col in columns:
        if row[col] == 1:
            sarcasm_types.append(col)
    return ",".join(sarcasm_types)

# Apply the function to each row and create a new column
isarcasm['sarcasm_type'] = isarcasm.apply(get_sarcasm_type, axis=1)

In [ ]:
isarcasm.head(10)

,Unnamed: 0,tweet,sarcastic,rephrase,sarcasm,irony,satire,understatement,overstatement,rhetorical_question,sarcasm_type
0,0,The only thing I got from college is a caffein...,1,"College is really difficult, expensive, tiring...",0.0,1.0,0.0,0.0,0.0,0.0,irony
1,1,I love it when professors draw a big question ...,1,I do not like when professors don’t write out ...,1.0,0.0,0.0,0.0,0.0,0.0,sarcasm
2,2,Remember the hundred emails from companies whe...,1,"I, at the bare minimum, wish companies actuall...",0.0,1.0,0.0,0.0,0.0,0.0,irony
3,3,Today my pop-pop told me I was not “forced” to...,1,"Today my pop-pop told me I was not ""forced"" to...",1.0,0.0,0.0,0.0,0.0,0.0,sarcasm
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,I would say Ted Cruz is an asshole and doesn’t...,1.0,0.0,0.0,0.0,0.0,0.0,sarcasm
5,5,"@jimrossignol I choose to interpret it as ""XD""...",1,It's a terrible name and the product sounds aw...,0.0,1.0,0.0,1.0,0.0,0.0,"irony,understatement"
6,6,Why would Alexa's recipe for Yorkshire pudding...,1,Great recipe from Alexa,0.0,1.0,0.0,0.0,0.0,1.0,"irony,rhetorical_question"
7,7,someone hit me w a horse tranquilizer istg ive...,1,Simply “I’m miserable.”,1.0,0.0,0.0,0.0,0.0,0.0,sarcasm
8,8,Loving season 4 of trump does America. Funnies...,1,this last year of trumps presidency is not goi...,1.0,0.0,0.0,0.0,0.0,0.0,sarcasm
9,9,Holly Arnold ??? Who #ImACeleb #MBE nope not ...,1,"Holly Arnold seem like a nice lady, just feel ...",1.0,0.0,0.0,0.0,0.0,1.0,"sarcasm,rhetorical_question"


In [ ]:
isarcasm['generated_sentence'] = 'No Sentence'

In [ ]:
def create_sentence(row):
    if row['generated_sentence'] == 'No Sentence':
        example = row['tweet']
        model = "meta-llama/Meta-Llama-3-8B-Instruct"
        if  row['sarcastic'] == 1:
            sarcasm_type = row['sarcasm_type']
            model = "meta-llama/Meta-Llama-3-8B-Instruct"
            messages = [{"role": "user", "content": f"You are a sarcastic twitter user. Create a sarcastic tweet similar to the example. The sarcasm should be of type: {sarcasm_type}. Respond only with the tweet. " +
                         f"Here is an example tweet to help you. Example: {example}"}]
            openai.api_base = "http://localhost:8000/v1"
            openai.api_key = "123"
            model_args={
                                "temperature": 0.7,
                                "max_tokens": 4000,
                                "top_p": 0.95,
                                "frequency_penalty": 0,
                                "presence_penalty": 0,
                            }

            try:
                response = openai.ChatCompletion.create(
                            model=model, messages=messages, request_timeout=60, **model_args
                        )

                return response['choices'][0]['message']['content']

            except Exception as e:  # Exception
                print(f"Error: {e}.")


                return "No Sentence"
        else:
            messages = [{"role": "user", "content": f"You are a twitter user. Create a tweet similar to the example. Respond only with the tweet. " +
                         f"Here is an example tweet to help you. Example: {example}"}]
            openai.api_base = "http://localhost:8000/v1"
            openai.api_key = "123"
            model_args={
                                "temperature": 0.7,
                                "max_tokens": 4000,
                                "top_p": 0.95,
                                "frequency_penalty": 0,
                                "presence_penalty": 0,
                            }

            try:
                response = openai.ChatCompletion.create(
                            model=model, messages=messages, request_timeout=60, **model_args
                        )

                return response['choices'][0]['message']['content']

            except Exception as e:  # Exception
                print(f"Error: {e}.")


                return "No Sentence"
    else:
        return row['generated_sentence']


In [ ]:
isarcasm['generated_sentence'] = DataFrameParallel(isarcasm, n_cores = 20, pbar = True).apply(create_sentence, axis=1)

Process-82: 100%|██████████| 174/174 [03:31<00:00,  1.22s/it]


In [ ]:
len(isarcasm[isarcasm['generated_sentence'] != 'No Sentence'])

3468

In [ ]:
generated_data = isarcasm[['generated_sentence','sarcastic','sarcasm_type']]

In [ ]:
generated_data

,generated_sentence,sarcastic,sarcasm_type
0,"""Wow, I'm so grateful to have finally reached ...",1,irony
1,"""Wow, I'm so grateful for the 10-page report t...",1,sarcasm
2,"""Wow, it's amazing how quickly companies jumpe...",1,irony
3,Just had the most enlightening conversation wi...,1,sarcasm
4,"""Wow, I'm so impressed that @BetoORourke is ac...",1,sarcasm
...,...,...,...
3463,"""Just got my morning coffee and I'm already dr...",0,
3464,I'm still stuck on the same 5th grade math hom...,0,
3465,Back from a whirlwind trip to Japan and feelin...,0,
3466,"""Huge congratulations to all the students who ...",0,


In [ ]:
generated_data.to_csv("isarcasm_synthetic_llama3_topp_95_temp_7.csv")